In [ ]:
import pandas as pd

# read data
kanja_mac_ctl = pd.read_csv('../preprocessed/data/kanja_mac_ctl.csv')
kanja_mac_ctl['観察終了年月'] = pd.to_datetime(kanja_mac_ctl['観察終了年月'], format='%Y%m')
kanja_summary = pd.read_csv('../preprocessed/data/kanja_summary.csv',parse_dates=['発症年月日'])
iyakuhin = pd.read_csv('../preprocessed/data/iyakuhin_mac_ctl.csv')
iyakuhin['診療年月'] = pd.to_datetime(iyakuhin['診療年月'], format='%Y%m')
iyakuhin['処方日'] = pd.to_datetime(iyakuhin['処方日'], format='%Y%m%d')

kanja = pd.merge(kanja_mac_ctl,kanja_summary[['加入者id','発症年月日']],on='加入者id',how='left')
data = pd.merge(iyakuhin,kanja[['加入者id','is_mac','発症年月日','観察終了年月']],on='加入者id',how='left')

data.loc[data.is_mac == 1,'month_maculopathy'] = (
           data['発症年月日'].dt.to_period('M').astype('int')
         - data['診療年月'].dt.to_period('M').astype('int')
)
data.loc[data.is_mac == 0,'month_maculopathy'] = (
           data['観察終了年月'].dt.to_period('M').astype('int')
         - data['診療年月'].dt.to_period('M').astype('int')
)

columns = data.columns.tolist()
new_col = ['加入者id','is_mac','month_maculopathy','医薬品コード','医薬品名']
other_col = [col for col in columns if col not in new_col]
new_col.extend(other_col)
data = data[new_col]

data.to_csv('../preprocessed/data/iyakuhin_mac_ctl_month.txt',index=False,sep='\t')
data
